In [1]:
import importlib
import mcp_server
importlib.reload(mcp_server)
from mcp_server import server
import asyncio


In [3]:
tools = await server()

CancelledError: 

In [ ]:
#tools[3].inputSchema["properties"]["passengers"]

{'type': 'array',
 'items': {'type': 'object',
  'properties': {'passengerType': {'type': 'string',
    'enum': ['ADT', 'CHD', 'INF', 'YAD', 'SRC', 'STD', 'CNS', 'STU'],
    'description': "Passenger type code. Common types: 'ADT' (Adult), 'CHD' (Child), 'INF' (Infant). Ensure to use a valid type from the allowed list."},
   'quantity': {'type': 'integer',
    'exclusiveMinimum': 0,
    'description': 'Number of passengers of this specific type. Must be a positive integer. Example: 1'}},
  'required': ['passengerType', 'quantity'],
  'additionalProperties': False,
  'description': 'Specifies the type and quantity of passengers.'},
 'minItems': 1,
 'description': 'An array specifying the number of passengers for each type (e.g., adults, children, infants).'}

In [ ]:
# her fonksiyonun inputSchema'sı var, bu inputSchema'da properties var
# bu propoerties fonksiyonun istediği parametereler
# her parametrenin enum kısmı kabul ettiği classları belirtiyor.
# her property bir description içeriyor.

In [15]:
tool_dict = {}

for tool in tools:
    tool_dict[tool.name] = {
        "description": tool.description,
    }
    if tool.inputSchema:
        tool_dict[tool.name]["inputSchema"] = {}
        for prop, details in tool.inputSchema["properties"].items():
            tool_dict[tool.name]["inputSchema"][prop] = {
                "description": details.get("description", "No description"),
                "enum": details.get("enum", None),
            }
    else:
        tool_dict[tool.name]["inputSchema"] = None

In [16]:
tool_dict

{'get_current_user_details': {'description': "Fetch member's profile, miles, identity.Tip: Call before other tools (i.e list_user_flights)",
  'inputSchema': {}},
 'get_expiring_miles': {'description': "Calculate member's expiring miles. Tip: Call after `get_current_user_details` to ensure member details are available.",
  'inputSchema': {}},
 'list_user_flights': {'description': "Fetch member's flights. Input: surname. Should be called after `get_current_user_details`. Tip: Call if the PNR has not yet been provided.",
  'inputSchema': {}},
 'search_flights': {'description': 'Search THY (Turkish Airlines) outbound flights. Based on information from user origin(s), destination(s), date(s), and passenger details to find available flights and their prices.Important: For round trips, ALWAYS call search_inbound_flights after selecting an outbound to get accurate total pricing and return options.',
  'inputSchema': {'originDestinations': {'description': "An array defining the flight segments

In [13]:
import asyncio
import sys
import json
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def get_structured_tools_async() -> list:
    """
    MCP sunucusuna bağlanır, kimlik doğrular ve sunucudaki tüm araçları,
    LangGraph için uygun, zengin ve detaylı bir yapıya dönüştürüp döndürür.
    """
    server_params = StdioServerParameters(
        command="npx",
        args=["-y", "mcp-remote", "https://mcp.turkishtechlab.com/mcp"],
    )

    with open("mcp_fetcher_errors.log", "w") as err_log:
        async with stdio_client(server_params, errlog=err_log) as (reader, writer):
            async with ClientSession(reader, writer) as session:
                await session.initialize()                
                tools_resp = await session.list_tools()
                langgraph_ready_tools = []

                for tool in tools_resp.tools:
                    tool_definition = {
                        "name": tool.name,
                        "description": tool.description or "",
                        "input_schema": tool.inputSchema or {},
                        "output_schema": tool.outputSchema or {}
                    }
                    langgraph_ready_tools.append(tool_definition)                
                return langgraph_ready_tools

In [14]:
await get_structured_tools_async()

[{'name': 'get_current_user_details',
  'description': "Fetch member's profile, miles, identity.Tip: Call before other tools (i.e list_user_flights)",
  'input_schema': {'type': 'object',
   'properties': {},
   'additionalProperties': False,
   '$schema': 'http://json-schema.org/draft-07/schema#'},
  'output_schema': {}},
 {'name': 'get_expiring_miles',
  'description': "Calculate member's expiring miles. Tip: Call after `get_current_user_details` to ensure member details are available.",
  'input_schema': {'type': 'object',
   'properties': {},
   'additionalProperties': False,
   '$schema': 'http://json-schema.org/draft-07/schema#'},
  'output_schema': {}},
 {'name': 'list_user_flights',
  'description': "Fetch member's flights. Input: surname. Should be called after `get_current_user_details`. Tip: Call if the PNR has not yet been provided.",
  'input_schema': {'type': 'object',
   'properties': {},
   'additionalProperties': False,
   '$schema': 'http://json-schema.org/draft-07/sc